In [0]:
import mlflow
import mlflow.spark
from pyspark.ml.recommendation import ALS

In [0]:
interactions = spark.table("test_catalog.default.als_interactions_30d")

als = ALS(
    userCol="customer_id",
    itemCol="product_id",
    ratingCol="interaction_weight",
    implicitPrefs=True,
    coldStartStrategy="drop",
    rank=20,
    maxIter=10,
    regParam=0.1
)

mlflow.set_experiment("/Workspace/Users/jung@ap-com.co.jp/mlops_demo/02. Modeling/")

with mlflow.start_run():
    model = als.fit(interactions)
    
    mlflow.spark.log_model(model, "als_model")

    mlflow.log_param("rank", 20)
    mlflow.log_param("maxIter", 10)
    mlflow.log_param("regParam", 0.1)